In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
HEADER = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}
url_list = [
    'https://movie.douban.com/people/98909222/wish?start=0&sort=time&rating=all&filter=all&mode=list',
    'https://movie.douban.com/people/98909222/wish?start=30&sort=time&rating=all&filter=all&mode=list'
]

In [3]:
def get_movie_info(movie_url):
    r_movie = requests.get(movie_url, headers=HEADER)
    soup_movie = BeautifulSoup(r_movie.text, 'html.parser')
    return {
        'name': str(soup_movie.find_all('span', {'property': 'v:itemreviewed'})[0].contents[0]),
        'year': int(soup_movie.find_all('span', {'class': 'year'})[0].contents[0].replace('(', '').replace(')', '')),
        'rating': float(soup_movie.find_all('strong', {'class': 'll rating_num', 'property': 'v:average'})[0].contents[0]),
        'votes': int(soup_movie.find_all('span', {'property': 'v:votes'})[0].contents[0])
    }

In [4]:
def get_page_info(url):
    r_page = requests.get(url, headers=HEADER)
    soup_page = BeautifulSoup(r_page.text, 'html.parser')  # 'lxml'
    movies = soup_page.find_all('div', {'class': 'title'})
    for i in range(0, len(movies)):
        movies_list.append(get_movie_info(movies[i].contents[1].attrs['href']))
        # print('No. ' + str(i))
    del i

In [5]:
movies_list = []
for i in range(0, len(url_list)):
    get_page_info(url_list[i])
del i

In [6]:
movies_df = pd.DataFrame(data=movies_list)
movies_df['rating_year'] = movies_df['rating'] * movies_df['year']
movies_df['rating_votes'] = movies_df['rating'] * movies_df['votes']
movies_df['rating_year_votes'] = movies_df['rating'] * movies_df['year'] * movies_df['votes']

In [7]:
movies_df.sort_values('rating_year_votes', ascending=False)

,name,year,rating,votes,rating_year,rating_votes,rating_year_votes
23,头号玩家 Ready Player One,2018,8.7,1032187,17556.6,8980026.9,1.812169e+10
46,教父 The Godfather,1972,9.3,682194,18339.6,6344404.2,1.251117e+10
45,教父2 The Godfather: Part Ⅱ,1974,9.2,378078,18160.8,3478317.6,6.866199e+09
37,社交网络 The Social Network,2010,8.1,365600,16281.0,2961360.0,5.952334e+09
11,雪国列车 설국열차,2013,7.5,332340,15097.5,2492550.0,5.017503e+09
9,E.T. 外星人 E.T.: The Extra-Terrestrial,1982,8.6,264147,17045.2,2271664.2,4.502438e+09
43,教父3 The Godfather: Part III,1990,8.9,252314,17711.0,2245594.6,4.468733e+09
38,华尔街之狼 The Wolf of Wall Street,2013,7.9,274278,15902.7,2166796.2,4.361761e+09
2,侏罗纪世界2 Jurassic World: Fallen Kingdom,2018,6.7,314789,13520.6,2109086.3,4.256136e+09
4,绿里奇迹 The Green Mile,1999,8.8,205235,17591.2,1806068.0,3.610330e+09
